# Prepare stuffs to show meditation and attention levels
* matplotlib to show signals in graph
* tqdm for progress bars to show attetion and meditation levels as gauges 
_(Note that we should use tqdm.notebook instead of tqdm to avoid new line in Jupyter notebook.)_

In [1]:
import matplotlib.pyplot as plt

%matplotlib widget
plt.rcParams['animation.html'] = 'jshtml'

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx ()

ax1.set_ylabel("Attention")
ax2.set_ylabel("Meditation")

ax1.set_ylim(0,100)
ax2.set_ylim(0,100)

fig.show()

from tqdm.notebook import tqdm

att = tqdm(total=100, desc="Attention level")
med = tqdm(total=100, desc="Meditation level")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Check and connect to device

### Show available devices
The headset normally show here with pattern ttyUSB*. In some case (in my case with Mac OS), it showed ttywch*.

In [2]:
!ls -ltrh /dev/tty* | grep dialout

crw-rw---- 1 root dialout 238,  2  2月 25 20:13 /dev/ttyTHS2
crw-rw---- 1 root dialout   4, 65  2月 25 20:13 /dev/ttyS1
crw-rw---- 1 root dialout   4, 67  2月 25 20:13 /dev/ttyS3
crw-rw---- 1 root dialout   4, 66  2月 25 20:13 /dev/ttyS2
crw-rw---- 1 root dialout 188,  0  2月 25 22:48 /dev/ttyUSB0


### Replace device and headset id
Replace device seen above and headset id. The headset id should be see the battery box.

In [3]:
import mindwave, time

In [4]:
headset = mindwave.Headset('/dev/ttyUSB0', '8C69')
time.sleep(2)

### Connect to the headset
After connecting, the led on the headset should change to blue color.

In [5]:
headset.connect()
print ("Connecting...")

while headset.status != 'connected':
    time.sleep(0.5)
    if headset.status == 'standby':
        headset.connect()
        print ("Retrying connect...")
print ("Connected.")

Connecting...
Connected.


## Process data

In [ ]:
i = 0

x, y_a, y_m = [], [], []

try:
    while True:
        attention = headset.attention
        meditation = headset.meditation
        
        # Update graphs
        x.append (i)
        y_a.append(attention)
        y_m.append(meditation)
        ax1.plot(x, y_a, color='r')
        ax2.plot(x, y_m, color='g')
        fig.canvas.draw()
        ax1.set_xlim(left=max(0, i-50), right=i+50)
        i += 1
        
        # Update progress bars
        med.update (meditation)
        att.update (attention)
        med.refresh ()
        att.refresh ()
        med.update (-meditation)
        att.update (-attention)
        
        time.sleep(.2)
        
except KeyboardInterrupt:
    pass


plt.close (fig)
att.close ()
med.close ()
headset.disconnect()
print ("Cleaned.")